<a href="https://colab.research.google.com/github/ti-molodykh/colab/blob/main/%D0%9D%D0%BE%D1%83%D1%82%D0%B1%D1%83%D0%BA_%D0%BA_%D0%B7%D0%B0%D0%BD%D1%8F%D1%82%D0%B8%D1%8E_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install psycopg2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Занятие 4

In [ ]:
import psycopg2
import pandas as pd

DB_HOST = 'de-edu-db.chronosavant.ru'
DB_USER = 'da11'
DB_USER_PASSWORD = 'gandalfthegrey'
DB_NAME = 'edu'

conn = psycopg2.connect(host=DB_HOST, user=DB_USER, password=DB_USER_PASSWORD, dbname=DB_NAME)

# Эффективность маркетинговых каналов

В хранилище имеются четыре таблицы:
- from da.user_visits – события посещения интернет-магазина за полгода
- from da.user_orders – события нажатия на кнопку "Checkout" за полгода
- from da.user_payments – события оплаты за заказ за полгода
- from da.user_x_channel – витрина с указанием канала привлечения для каждого идентификатор юзера

Нам нужно посчитать следующие метрики:
1. **количество уникальных визитов:** сколько людей привлекли?
2. **количество добавивших в корзину:** сколько из них добавили в корзину?
3. **CR to cart:** какая вероятность добавления в корзину?
4. **количество заплативших:** сколько из них заплатили?
5. **CR to payment:** какая вероятность оплаты?
6. **ARPPU:** сколько заработали в указанный день с одного платящего?
7. **траты:** сколько денег потратили на канал?
8. **CPC:** сколько стоит один платящий?
9. **LTV:** сколько принесет платящий за все время?

Алгоритм действий:
1. Считаем метрики 1-6
2. Выносим результаты в google sheets – 2 вкладка: (https://docs.google.com/spreadsheets/d/1IJggqC7o4sg19mrWHNH6DCX1pQlwzOK0X75Y4bgVFmw/edit#gid=175543509)
3. Дополняем результаты в google sheets метриками "Траты" и "CPC" 
4. Возвращаемся в ноутбук считать LTV 

# Метрики 1-6

In [ ]:
pd.read_sql_query("""
select channel, 'CR from Visit to Order' as metric_name, (count(distinct uo.user_id)::float/count(distinct uv.user_id)::float) metric_value,
count(distinct uo.user_id),
count(distinct uv.user_id)
from da.user_visits uv
left join da.user_orders uo using(user_id)
left join da.user_x_channel using(user_id)
group by 1 , 2
""", conn)

,channel,metric_name,metric_value,count,count
0,google direct,CR from Visit to Order,0.038931,102,2620
1,Organic desktop,CR from Visit to Order,0.011685,134,11468
2,Organic mobile,CR from Visit to Order,0.012395,25,2017
3,vk.com,CR from Visit to Order,0.124734,644,5163
4,yandex direct,CR from Visit to Order,0.011792,41,3477


In [ ]:
conn = psycopg2.connect(host=DB_HOST, user=DB_USER, password=DB_USER_PASSWORD, dbname=DB_NAME)

stats = pd.read_sql_query("""
/*
--количество уникальных визитов: сколько людей привлекли?
--количество добавивших в корзину: сколько из них добавили в корзину?
--количество заплативших: сколько из них заплатили?
--CR to cart: какая вероятность добавления в корзину?
--CR to payment: какая вероятность оплаты?
ARPPU: сколько заработали в указанный день с одного платящего?

from da.user_visits – события посещения интернет-магазина за полгода
from da.user_orders – события нажатия на кнопку "Checkout" за полгода
from da.user_payments – события оплаты за заказ за полгода
from da.user_x_channel – витрина с указанием канала привлечения для каждого идентификатор юзера
*/

select
channel, 'Unique visitors' as metrics, count(distinct uv.user_id) as metric_value
from da.user_visits uv
left join da.user_x_channel x
on uv.user_id = x.user_id
group by channel, metrics
union all
select
channel, 'Unique visitors with orders' as metrics, count(distinct uo.user_id) as metric_value
from da.user_orders uo
left join da.user_x_channel x
on uo.user_id = x.user_id
group by channel, metrics
union all
select
channel, 'Unique visitors with payment' as metrics, count(distinct up.user_id) as metric_value
from da.user_payments up
left join da.user_x_channel x
on up.user_id = x.user_id
group by channel, metrics
union all
select
channel, 'CR from Visit to Order' as metrics, cast(count(distinct uo.user_id) as decimal(23,5))/cast(count(distinct uv.user_id) as decimal(23,5)) as metric_value
from da.user_visits uv
left join da.user_x_channel x
on uv.user_id = x.user_id
left join da.user_orders uo
on uv.user_id = uo.user_id
group by channel, metrics
union all
select
channel, 'CR from Visit to Payment' as metrics, cast(count(distinct up.user_id) as decimal(23,5))/cast(count(distinct uv.user_id) as decimal(23,5)) as metric_value
from da.user_visits uv
left join da.user_x_channel x
on uv.user_id = x.user_id
left join da.user_payments up
on uv.user_id = up.user_id
group by channel, metrics
union all
select
channel, 'ARPPU' as metrics, sum(amount-amount_refunded)/count(distinct case when amount-amount_refunded > 0 then up.user_id else null end) as metric_value
from da.user_payments up
left join da.user_x_channel x
on up.user_id = x.user_id
group by channel, metrics
""", conn)

In [ ]:
stats

,channel,metrics,metric_value
0,google direct,Unique visitors,2620.000000
1,Organic desktop,Unique visitors,11468.000000
2,Organic mobile,Unique visitors,2017.000000
3,vk.com,Unique visitors,5163.000000
4,yandex direct,Unique visitors,3477.000000
5,google direct,Unique visitors with orders,102.000000
6,Organic desktop,Unique visitors with orders,134.000000
7,Organic mobile,Unique visitors with orders,25.000000
8,vk.com,Unique visitors with orders,644.000000
9,yandex direct,Unique visitors with orders,41.000000


# LTV

In [ ]:
ltv = pd.read_sql_query("""
-- Делим юзеров по когортам: дата первого платежа
with user_cohort as (
  select user_id, min(created) cohort
  from da.user_payments
  group by 1 
),
cum_user_revenue as (
  -- по кжадому каналу и юзеру считаем сколько трат у него был к N-му месяцу после первого платежа 
  -- LTV привязан к месяцу: к 1-му месяцу юзер может потратить 100 рублей, а к 12-му 10 000 руб, тк возвращается и делает повторные заказы
select 
  channel, 
  user_id, 
  DATE_PART('month', created::date) - DATE_PART('month', cohort::date) m, 
  sum(amount - amount_refunded) over (partition by user_id order by DATE_PART('month', created::date) - DATE_PART('month', cohort::date) ) as cum_sum
from da.user_payments
join user_cohort using(user_id)
left join da.user_x_channel using(user_id)
order by 2, 3
)
select channel, m, avg(cum_sum) ltv
from cum_user_revenue
group by 1, 2
order by 1, 2
""", conn)
ltv

,channel,m,ltv
0,google direct,0.0,5257.108696
1,google direct,1.0,15217.458333
2,google direct,2.0,6405.071429
3,google direct,3.0,7215.454545
4,google direct,4.0,9537.142857
5,google direct,5.0,10493.000000
6,Organic desktop,0.0,1859.454545
7,Organic desktop,1.0,2998.666667
8,Organic desktop,2.0,4497.000000
9,Organic desktop,3.0,5996.000000
